In [1]:
from use_translator import *

In [131]:
def merge_answer(results,qnode_ids,p=False):
    #We are going rename most of the node and edge bindings.  But, we want to preserve bindings to things in the
    # original query.  For the current creative one-hop, that is just nodes b/c we are replacing the one edge.
    #The original qnode bindings are the same in all results by construction
    mergedresult = {'node_bindings': { q: results[0]['node_bindings'][q] for q in qnode_ids},
                    'edge_bindings': {}}
    #if 'PUBCHEM.COMPOUND:6623' in qnode_ids:
    #    print('ooo')
    if p:
        for r in results:
            print('x')
    for bindingtype in ['node', 'edge']:
        bound_things = set()
        for result in results:
            # the gross part here is dealing with the lists in the values of the bindings.
            for key,binding in result[f'{bindingtype}_bindings'].items():
                if key in qnode_ids:
                    continue
                bound = frozenset([x['id'] for x in binding])
                if bound not in bound_things:
                    #found a binding to add
                    n = f'_dummy_{bindingtype}_{len(bound_things)}'
                    mergedresult[f'{bindingtype}_bindings'][n] = binding
                    bound_things.add(bound)
    if p:
        print(mergedresult['node_bindings'])
    return mergedresult

from copy import deepcopy
#TODO move into operations? Make a translator op out of this
def merge_results_by_node(result_message, merge_qnode):
    """This assumes a single result message, with a single merged KG.  The goal is to take all results that share a
    binding for merge_qnode and combine them into a single result.
    Assumes that the results are not scored."""
    #This is relatively straightforward: group all the results by the merge_qnode
    # for each one, the only complication is in the keys for the "dummy" bindings.
    original_results = result_message['message']['results']
    #original_qnodes = result_message['message']['query_graph']['nodes'].keys()
    #hackhack
    original_qnodes = set(['disease','chemical'])
    #print(original_qnodes)
    #group results
    grouped_results = defaultdict(list)
    for result in original_results:
        answer = result['node_bindings'][merge_qnode]
        bound = frozenset([x['id'] for x in answer])
        if 'PUBCHEM.COMPOUND:6623' in bound:
            print(result['node_bindings'])
        grouped_results[bound].append(result)
    #TODO : I'm sure there's a better way to handle this with asyncio
    new_results = []
    for r in grouped_results:
        p = False
        x = merge_answer(grouped_results[r],original_qnodes,p)
        new_results.append(x)
    return_message = deepcopy(result_message)
    return_message['message']['results'] = new_results
    return return_message


In [109]:
def create_query(disease_id, n_named_things=1):
    query_graph = {'nodes': {'disease': {'ids': [disease_id]}},
                   'edges': {}}
    pnode = 'disease'
    for i in range(n_named_things):
        cnode = f'nt_{i}'
        query_graph['nodes'][cnode] = {'categories':['biolink:NamedThing']}
        query_graph['edges'][f'edge_{i}'] = {'subject': pnode, 'object': cnode}
        pnode = cnode
    cnode = 'chemical'
    query_graph['nodes'][cnode] = {'categories': ['biolink:ChemicalEntity']}
    query_graph['edges']['dedge'] = {'subject': pnode, 'object': cnode}
    #message = {'message':{'query_graph': query_graph}, 'workflow':[{'id':'lookup'}]}
    message = {'message':{'query_graph': query_graph}}
    return message

In [110]:
q1 =  create_query('MONDO:0008078',1) 

In [72]:
#original_result = post('local_robokop','http://localhost:4869/robokop/query',q1)

local_robokop returned in 0:04:40.240870s
Num Results: 5179


In [113]:
original_result = automat('robokopkg',q1)

automat/robokopkg returned in 0:04:46.391894s
Num Results: 5179


In [119]:
for r in original_result['message']['results']:
    s = json.dumps(r)
    if 'NCBIGene' in s:
        print(r['node_bindings']['chemical'])

[{'id': 'PUBCHEM.COMPOUND:159832'}]
[{'id': 'PUBCHEM.COMPOUND:409301'}]
[{'id': 'PUBCHEM.COMPOUND:6623'}]
[{'id': 'PUBCHEM.COMPOUND:2391'}]
[{'id': 'PUBCHEM.COMPOUND:449459'}]
[{'id': 'PUBCHEM.COMPOUND:5460341'}]


In [127]:
merged = merge_results_by_node(original_result,'chemical')

{'disease': [{'id': 'MONDO:0008078', 'qnode_id': 'MONDO:0008078'}], 'nt_0': [{'id': 'MONDO:0005328'}], 'chemical': [{'id': 'PUBCHEM.COMPOUND:6623'}]}
{'disease': [{'id': 'MONDO:0008078', 'qnode_id': 'MONDO:0008078'}], 'nt_0': [{'id': 'MONDO:0004995'}], 'chemical': [{'id': 'PUBCHEM.COMPOUND:6623'}]}
{'disease': [{'id': 'MONDO:0008078', 'qnode_id': 'MONDO:0008078'}], 'nt_0': [{'id': 'MONDO:0005044'}], 'chemical': [{'id': 'PUBCHEM.COMPOUND:6623'}]}
{'disease': [{'id': 'MONDO:0008078', 'qnode_id': 'MONDO:0008078'}], 'nt_0': [{'id': 'MONDO:0005240'}], 'chemical': [{'id': 'PUBCHEM.COMPOUND:6623'}]}
{'disease': [{'id': 'MONDO:0008078', 'qnode_id': 'MONDO:0008078'}], 'nt_0': [{'id': 'NCBIGene:4763'}], 'chemical': [{'id': 'PUBCHEM.COMPOUND:6623'}]}
{'disease': [{'id': 'MONDO:0008078', 'qnode_id': 'MONDO:0008078'}], 'nt_0': [{'id': 'MONDO:0000839'}], 'chemical': [{'id': 'PUBCHEM.COMPOUND:6623'}]}
{'disease': [{'id': 'MONDO:0008078', 'qnode_id': 'MONDO:0008078'}], 'nt_0': [{'id': 'MONDO:0005385'}

In [128]:
merged['workflow']=[{'id':'overlay_connect_knodes'},{'id':'score'}]

In [129]:
for r in merged['message']['results']:
    s = json.dumps(r)
    if 'NCBIGene' in s:
        print(r['node_bindings']['chemical'])

[{'id': 'PUBCHEM.COMPOUND:6623'}]
[{'id': 'PUBCHEM.COMPOUND:409301'}]
[{'id': 'PUBCHEM.COMPOUND:2391'}]
[{'id': 'PUBCHEM.COMPOUND:5460341'}]
[{'id': 'PUBCHEM.COMPOUND:159832'}]
[{'id': 'PUBCHEM.COMPOUND:449459'}]


In [130]:
result = post('local_robokop','http://localhost:4869/robokop/query',merged)

local_robokop returned in 0:02:06.435638s
Num Results: 2398


In [132]:
with open('twohop_nf.json','w') as outf:
    json.dump(result,outf,indent=4)

In [134]:
for n,r in enumerate(result['message']['results']):
    s = json.dumps(r)
    if 'NCBIGene' in s:
        print(n)
        print(r['node_bindings']['chemical'])
        cid = r['node_bindings']['chemical'][0]['id']
        print(cid)

159
[{'id': 'PUBCHEM.COMPOUND:449459'}]
PUBCHEM.COMPOUND:449459
1450
[{'id': 'PUBCHEM.COMPOUND:5460341'}]
PUBCHEM.COMPOUND:5460341
2070
[{'id': 'PUBCHEM.COMPOUND:159832'}]
PUBCHEM.COMPOUND:159832
2144
[{'id': 'PUBCHEM.COMPOUND:2391'}]
PUBCHEM.COMPOUND:2391
2333
[{'id': 'PUBCHEM.COMPOUND:409301'}]
PUBCHEM.COMPOUND:409301
2397
[{'id': 'PUBCHEM.COMPOUND:6623'}]
PUBCHEM.COMPOUND:6623


In [135]:
q2 =  create_query('MONDO:0008078',2) 

In [136]:
original_result_2 = automat('robokopkg',q2)

KeyboardInterrupt: 